In [1]:
import pandas as pd
import numpy as np
import boto3
import io

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

### Import Data

In [2]:
bucket = "nrel-vizstock"
file_name = "doe/2022/res_full_01_2018_550k_20220725/metadata/baseline.parquet"

s3 = boto3.client('s3') 
obj = s3.get_object(Bucket= bucket, Key= file_name) 
pqt = pd.read_parquet(io.BytesIO(obj['Body'].read()))

# import results parquet
# pqt = pd.read_parquet('metadata.parquet')

In [3]:
# parse timeseries and metadata
pqt_ts = pqt.loc[:, pqt.columns.str.startswith('out.')]
pqt_meta = pqt.loc[:, pqt.columns.str.startswith('in.')]

other_cols = list(set(pqt.columns) - set(pqt_meta.columns) - set(pqt_ts.columns))
pqt_other = pqt.loc[:, other_cols]

### Metadata

In [4]:
# Checks if string is a float number
def isfloat(string):
    try:
        float(string)
        return True
    except:
        return False  

# Converts string values to int or float values if possible
def convert_to_numeric(s):
    if not type(s) == str:
        return s
    
    if s.isnumeric():
        return int(s)
    elif isfloat(s):
        return float(s)
    else:
        return s

In [5]:
# Enumerate values and convert to numeric if possible
enums = {col: [convert_to_numeric(s) for s in sorted(pqt_meta[col].unique())] for col in pqt_meta.columns}
enums_list = ['|'.join([str(elem) for elem in enum]) if enum is not None else None for enum in list(enums.values())] #format

# Store datatypes for each column
dtypes = {k: str(pd.Series(v).dtypes) for k, v in enums.items()}

In [6]:
df_meta = pd.DataFrame({'field_location': 'metadata', 
                        'field_name': pqt_meta.columns, 
                        'data_type': list(dtypes.values()),
                        'allowable_enumerations': enums_list})

dtype_map = {
    'float64': 'float',
    'int64': 'integer',
    'object': 'string',
    'bool': 'bool'
}

df_meta['data_type'] = df_meta['data_type'].map(dtype_map)
df_meta.loc[df_meta['data_type'] == 'bool', 'allowable_enumerations'] = 'True|False'

In [7]:
df_meta.head()

,field_location,field_name,data_type,allowable_enumerations
0,metadata,in.sqft,float,317.0|328.0|333.0|617.0|633.0|853.0|866.0|885.0|1138.0|1202.0|1220.0|1623.0|1675.0|1690.0|2115.0|2152.0|2176.0|2590.0|2631.0|2663.0|3138.0|3241.0|3301.0|8194.0|12291.0|13414.0
1,metadata,in.ahs_region,string,"CBSA Atlanta-Sandy Springs-Roswell, GA|CBSA Boston-Cambridge-Newton, MA-NH|CBSA Chicago-Naperville-Elgin, IL-IN-WI|CBSA Dallas-Fort Worth-Arlington, TX|CBSA Detroit-Warren-Dearborn, MI|CBSA Houston-The Woodlands-Sugar Land, TX|CBSA Los Angeles-Long Beach-Anaheim, CA|CBSA Miami-Fort Lauderdale-West Palm Beach, FL|CBSA New York-Newark-Jersey City, NY-NJ-PA|CBSA Philadelphia-Camden-Wilmington, PA-NJ-DE-MD|CBSA Phoenix-Mesa-Scottsdale, AZ|CBSA Riverside-San Bernardino-Ontario, CA|CBSA San Francisco-Oakland-Hayward, CA|CBSA Seattle-Tacoma-Bellevue, WA|CBSA Washington-Arlington-Alexandria, DC-VA-MD-WV|Non-CBSA East North Central|Non-CBSA East South Central|Non-CBSA Middle Atlantic|Non-CBSA Mountain|Non-CBSA New England|Non-CBSA Pacific|Non-CBSA South Atlantic|Non-CBSA West North Central|Non-CBSA West South Central"
2,metadata,in.ashrae_iecc_climate_zone_2004,string,1A|2A|2B|3A|3B|3C|4A|4B|4C|5A|5B|6A|6B|7A|7B
3,metadata,in.ashrae_iecc_climate_zone_2004_2_a_split,string,"1A|2A - FL, GA, AL, MS|2A - TX, LA|2B|3A|3B|3C|4A|4B|4C|5A|5B|6A|6B|7A|7B"
4,metadata,in.bathroom_spot_vent_hour,string,Hour0|Hour1|Hour10|Hour11|Hour12|Hour13|Hour14|Hour15|Hour16|Hour17|Hour18|Hour19|Hour2|Hour20|Hour21|Hour22|Hour23|Hour3|Hour4|Hour5|Hour6|Hour7|Hour8|Hour9


### Timeseries

In [8]:
df_ts = pd.DataFrame({'field_location': 'timeseries', 
                        'field_name': pqt_ts.columns, 
                        'data_type': [str(dtype) for dtype in pqt_ts.dtypes.values],
                        'allowable_enumerations': None})

df_ts['data_type'] = df_ts['data_type'].map(dtype_map)

### Data dictionary

In [9]:
df_data = pd.concat([df_meta, df_ts]).reset_index(drop=True)
df_data['allowable_enumerations'] = df_data['allowable_enumerations'].fillna('n/a')

In [10]:
# Merge in units and descriptions
df_descriptions = pd.read_excel('data_description_units.xlsx', engine='openpyxl')
df_data = df_data.merge(df_descriptions, 'outer',  on='field_name')

cols = ['field_location', 'field_name', 'data_type', 'units', 'field_description', 'allowable_enumerations']
df_data = df_data[cols]
df_data = df_data.fillna('n/a')

In [11]:
df_data.head()

,field_location,field_name,data_type,units,field_description,allowable_enumerations
0,metadata,in.sqft,float,sqft,Finished floor area of the housing unit,317.0|328.0|333.0|617.0|633.0|853.0|866.0|885.0|1138.0|1202.0|1220.0|1623.0|1675.0|1690.0|2115.0|2152.0|2176.0|2590.0|2631.0|2663.0|3138.0|3241.0|3301.0|8194.0|12291.0|13414.0
1,metadata,in.ahs_region,string,n/a,American Housing Survey region,"CBSA Atlanta-Sandy Springs-Roswell, GA|CBSA Boston-Cambridge-Newton, MA-NH|CBSA Chicago-Naperville-Elgin, IL-IN-WI|CBSA Dallas-Fort Worth-Arlington, TX|CBSA Detroit-Warren-Dearborn, MI|CBSA Houston-The Woodlands-Sugar Land, TX|CBSA Los Angeles-Long Beach-Anaheim, CA|CBSA Miami-Fort Lauderdale-West Palm Beach, FL|CBSA New York-Newark-Jersey City, NY-NJ-PA|CBSA Philadelphia-Camden-Wilmington, PA-NJ-DE-MD|CBSA Phoenix-Mesa-Scottsdale, AZ|CBSA Riverside-San Bernardino-Ontario, CA|CBSA San Francisco-Oakland-Hayward, CA|CBSA Seattle-Tacoma-Bellevue, WA|CBSA Washington-Arlington-Alexandria, DC-VA-MD-WV|Non-CBSA East North Central|Non-CBSA East South Central|Non-CBSA Middle Atlantic|Non-CBSA Mountain|Non-CBSA New England|Non-CBSA Pacific|Non-CBSA South Atlantic|Non-CBSA West North Central|Non-CBSA West South Central"
2,metadata,in.ashrae_iecc_climate_zone_2004,string,n/a,IECC climate zone,1A|2A|2B|3A|3B|3C|4A|4B|4C|5A|5B|6A|6B|7A|7B
3,metadata,in.ashrae_iecc_climate_zone_2004_2_a_split,string,n/a,n/a,"1A|2A - FL, GA, AL, MS|2A - TX, LA|2B|3A|3B|3C|4A|4B|4C|5A|5B|6A|6B|7A|7B"
4,metadata,in.bathroom_spot_vent_hour,string,n/a,Bathroom spot ventilation daily start hour,Hour0|Hour1|Hour10|Hour11|Hour12|Hour13|Hour14|Hour15|Hour16|Hour17|Hour18|Hour19|Hour2|Hour20|Hour21|Hour22|Hour23|Hour3|Hour4|Hour5|Hour6|Hour7|Hour8|Hour9


In [12]:
df_data.to_csv('data_dictionary.tsv', sep='\t', index=False)

### Enumeration Dictionary

In [13]:
# Output template with all enumerations
enum_dict = pd.DataFrame()
for col, enum_list in enums.items():
    df = pd.DataFrame(enum_list, index=[col]*len(enum_list), columns=['enumeration'])
    enum_dict = enum_dict.append(df)

enum_dict['enumeration_description'] = None
enum_dict.to_csv('enumeration_dictionary_temp.tsv', sep='\t')

C:\Users\pwhite2\AppData\Local\Temp\1\ipykernel_34076\1512421267.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  enum_dict = enum_dict.append(df)
C:\Users\pwhite2\AppData\Local\Temp\1\ipykernel_34076\1512421267.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  enum_dict = enum_dict.append(df)
C:\Users\pwhite2\AppData\Local\Temp\1\ipykernel_34076\1512421267.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  enum_dict = enum_dict.append(df)
C:\Users\pwhite2\AppData\Local\Temp\1\ipykernel_34076\1512421267.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  enum_dict = enum_dict.append(df)
C:\Users\pwhite2\AppData\Local\Temp\1\ipyker

In [2]:
# Save manually updated dictionary as tsv
df = pd.read_excel('enumeration_dictionary.xlsx', engine='openpyxl')
df.to_csv('enumeration_dictionary.tsv', sep='\t')